# 20220518

开始第三版带有晶界模型的测试

首先生成一个210晶界模型


In [ ]:
from kmcpy.event_generator import *
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN

    
def build_distance_matrix_from_getnninfo_output(cutoffdnn_output={},verbose=False):
    """build a distance matrix from the output of CutOffDictNN.get_nn_info

    nn_info looks like: 
    [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
    
    or say:
    
    nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
    
    Use the site.distance function to build matrix
    

    Args:
        cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

    Returns:
        np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
    """

    distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
        

    for sitedictindex1 in range(0,len(cutoffdnn_output)):
        for sitedictindex2 in range(0,len(cutoffdnn_output)):
            """Reason for jimage=[0,0,0]
            
            site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

            .
            """
            distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
        
        
    
    return distance_matrix

li3clo_210=Structure.from_file("CONTCAR")
li3clo_210.to("cif","210.cif")
li3clo_210.add_oxidation_state_by_guess()

#local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","O2-"):4.0})
local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","Li+"):3.0})
#local_env_finder = CutOffDictNN({("Li+","Cl-"):2.8})
np.set_printoptions(precision=2,suppress=True)  
site_to_mark1=[]
site_to_mark2=[]
site_to_mark3=[]
for i in range(0,len(li3clo_210)):
    if "Li+" not in li3clo_210[i].species:
        continue
    
    #print(local_env_finder.get_nn_info(li3clo_210,i))
    print(len(local_env_finder.get_nn_info(li3clo_210,i)))
    """
    
    for #local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","O2-"):4.0})
        if len(local_env_finder.get_nn_info(li3clo_210,i))==5:
        print(local_env_finder.get_nn_info(li3clo_210,i))
        li3clo_210.replace(i,{"Na+":1.0})

    if len(local_env_finder.get_nn_info(li3clo_210,i))==4:
        print(local_env_finder.get_nn_info(li3clo_210,i))
        li3clo_210.replace(i,{"K+":1.0})
    """
    
    if len(local_env_finder.get_nn_info(li3clo_210,i))!=12:
        print(local_env_finder.get_nn_info(li3clo_210,i))
        print(build_distance_matrix_from_getnninfo_output(local_env_finder.get_nn_info(li3clo_210,i)))
        site_contain_li=[neighbor["site"].species for neighbor in local_env_finder.get_nn_info(li3clo_210,i)]
        li_num=0
        for site in site_contain_li:
            if "Li+" in site:
                li_num+=1
        print("Li num",li_num)

        if len(local_env_finder.get_nn_info(li3clo_210,i))==11:
            site_to_mark1.append(i)
        elif len(local_env_finder.get_nn_info(li3clo_210,i))==8:
            site_contain_li=[neighbor["site"].species for neighbor in local_env_finder.get_nn_info(li3clo_210,i)]
            li_num=0
            for site in site_contain_li:
                if "Li+" in site:
                    li_num+=1
            if li_num==6:
                site_to_mark2.append(i)
            else:
                site_to_mark3.append(i)
        else:
            raise ValueError()

for i in site_to_mark1:
    li3clo_210.replace(i,{"Na+":1.0})
for i in site_to_mark2:
    li3clo_210.replace(i,{"K+":1.0})
for i in site_to_mark3:
    li3clo_210.replace(i,{"Rb+":1.0})
li3clo_210.to("cif","marked210.cif")
    
    #print(local_env_finder.get_all_nn_info(li3clo_210))



For 210 grain boundary, there are several types of Li environment

1. General Lithium (larger green atom): 8 Li+ and 4 Cl- and 2 O2-
2. Gold: 7 Li+ and 4 Cl- and 2O2-
3. Pink: 4 Li+ and 4 Cl- and 1O2-
4. Purple: 6 Li+ and 2 Cl-  and 2O2-

**One important thing is that the distance matrix are different for type 2,3,4**

Starting initialize a new function to generate different kinds of clusters


In [21]:
from kmcpy.event_generator import *
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN

    
def build_distance_matrix_from_getnninfo_output(cutoffdnn_output={},verbose=False):
    """build a distance matrix from the output of CutOffDictNN.get_nn_info

    nn_info looks like: 
    [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
    
    or say:
    
    nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
    
    Use the site.distance function to build matrix
    

    Args:
        cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

    Returns:
        np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
    """

    distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
        

    for sitedictindex1 in range(0,len(cutoffdnn_output)):
        for sitedictindex2 in range(0,len(cutoffdnn_output)):
            """Reason for jimage=[0,0,0]
            
            site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

            .
            """
            distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
        
        
    
    return distance_matrix



li3clo_210=Structure.from_file("CONTCAR")
li3clo_210.add_oxidation_state_by_guess()
#local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","O2-"):4.0})
local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","Li+"):3.0})

clusters=[]



for i in range(0,len(li3clo_210)):
    if "Li+" in li3clo_210[i].species:
        nn=local_env_finder.get_nn_info(li3clo_210,i)
        nn_elements=[neighbor["site"].species.elements[0] for neighbor in nn]
        #print(local_env_finder.get_cn_dict(li3clo_210,i))
        
        if local_env_finder.get_cn_dict(li3clo_210,i) not in clusters:
            clusters.append(local_env_finder.get_cn_dict(li3clo_210,i))
print(clusters)

[{'Li+': 8, 'Cl-': 4}, {'Li+': 7, 'Cl-': 4}, {'Cl-': 4, 'Li+': 4}, {'Li+': 6, 'Cl-': 2}]


# start new attempt on rearranging neighbor
it is successful
mainly for accelerate the brutely find neighbors process

In [32]:
import numpy as np

import json
import logging
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN
import itertools

def build_distance_matrix_from_getnninfo_output(cutoffdnn_output=[],verbose=False):
    """build a distance matrix from the output of CutOffDictNN.get_nn_info

    nn_info looks like: 
    [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
    
    or say:
    
    nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
    
    Use the site.distance function to build matrix
    

    Args:
        cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

    Returns:
        np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
    """

    distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
        

    for sitedictindex1 in range(0,len(cutoffdnn_output)):
        for sitedictindex2 in range(0,len(cutoffdnn_output)):
            """Reason for jimage=[0,0,0]
            
            site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

            .
            """
            distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
        
        
    
    return distance_matrix


nasicon=Structure.from_cif("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",primitive=True)

center_Na1=[0,1]
local_env_finder = CutOffDictNN({('Na+','Na+'):4,('Na+','Si4+'):4})


reference_neighbor_sequences=sorted(sorted(local_env_finder.get_nn_info(nasicon,center_Na1[0]),key=lambda x:x["wyckoff_sequence"]),key = lambda x:x["label"])     

reference_distance_matrix=build_distance_matrix_from_getnninfo_output(reference_neighbor_sequences)

print(reference_neighbor_sequences)
np.set_printoptions(precision=2,suppress=True)

reference_na_distance_matrix=build_distance_matrix_from_getnninfo_output(reference_neighbor_sequences[0:6])


possible_arrange_na=[]

for possible_na_sequence in itertools.permutations(reference_neighbor_sequences[0:6]):
    if np.allclose(build_distance_matrix_from_getnninfo_output(possible_na_sequence),reference_na_distance_matrix,rtol=0.01):
        possible_arrange_na.append(possible_na_sequence)
        
print(len(possible_arrange_na))

reference_si_distance_matrix=build_distance_matrix_from_getnninfo_output(reference_neighbor_sequences[6:12])


possible_arrange_si=[]

for possible_si_sequence in itertools.permutations(reference_neighbor_sequences[6:12]):
    if np.allclose(build_distance_matrix_from_getnninfo_output(possible_si_sequence),reference_si_distance_matrix,rtol=0.01):
        possible_arrange_si.append(possible_si_sequence)

print(len(possible_arrange_si))


total_iter=0

for possible_complete_sequence in itertools.product(possible_arrange_na,possible_arrange_si):
    re_sorted_neighbors_list=[]
    
    for neighbor in possible_complete_sequence:

        re_sorted_neighbors_list.extend(list(neighbor))     

    if np.allclose(build_distance_matrix_from_getnninfo_output(re_sorted_neighbors_list),reference_distance_matrix,rtol=0.01):
        print(possible_complete_sequence)
        total_iter+=1
    pass

print(total_iter)
        



[{'site': PeriodicSite: Na+ (1.2831, 2.6519, 12.9389) [0.8897, 0.6103, 0.2500], 'image': (0, 0, 0), 'weight': 3.4778161424304033, 'site_index': 2, 'wyckoff_sequence': 0, 'local_index': 18, 'label': 'Na2'}, {'site': PeriodicSite: Na+ (1.6550, -2.4371, 12.9389) [0.6103, 0.2500, 0.8897], 'image': (0, 0, 0), 'weight': 3.4778161424304033, 'site_index': 3, 'wyckoff_sequence': 1, 'local_index': 19, 'label': 'Na2'}, {'site': PeriodicSite: Na+ (-2.9381, -0.2148, 12.9389) [0.2500, 0.8897, 0.6103], 'image': (0, 0, 0), 'weight': 3.477816142430404, 'site_index': 4, 'wyckoff_sequence': 2, 'local_index': 20, 'label': 'Na2'}, {'site': PeriodicSite: Na+ (-1.6550, 2.4371, 9.2421) [0.3897, 0.7500, 0.1103], 'image': (0, 0, 0), 'weight': 3.4778161424304015, 'site_index': 5, 'wyckoff_sequence': 3, 'local_index': 21, 'label': 'Na2'}, {'site': PeriodicSite: Na+ (2.9381, 0.2148, 9.2421) [0.7500, 0.1103, 0.3897], 'image': (0, 0, 0), 'weight': 3.4778161424304006, 'site_index': 6, 'wyckoff_sequence': 4, 'local_in

# generate events version 3

In [1]:
import numpy as np
import json
import logging
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN
import itertools
from kmcpy.io import convert
from kmcpy.event import Event
from kmcpy.event_generator import *



np.set_printoptions(precision=2,suppress=True)

    
generate_events3(prim_cif_name="210.cif",local_env_cutoff_dict={("Li+","Cl-"):4.0,("Li+","Li+"):3.0},atom_identifier_type="specie",center_atom_identifier="Li+",diffuse_to_atom_identifier="Li+",species_to_be_removed=["O2-","O"],distance_matrix_rtol=0.25,distance_matrix_atol=0.5,find_nearest_if_fail=True,convert_to_primitive_cell=False,create_reference_cluster=True,supercell_shape=[2,1,1],event_fname="events.json",event_kernal_fname='event_kernal.csv',verbosity=0)
"""
def convert2(o):
    if isinstance(o, np.int64): return int(o)
    elif isinstance(o, np.int32): return int(o)  
    try:
        return o.as_dict()
    except:
        return o


with open("primitive_cell_neighbor.json","w") as f:
    jsonstr=json.dumps(primitive_cell_cutoff_dict,indent=4,default=convert2)
    f.write(jsonstr) 
"""

INFO:root:Li+120 Cl-40
INFO:root:please check if these are center atom:
INFO:root:[1.92 3.51 6.43] Li+
INFO:root:[ 1.93  3.5  15.15] Li+
INFO:root:[ 1.93  3.53 23.85] Li+
INFO:root:[ 1.93  3.72 32.69] Li+
INFO:root:[1.93 0.05 4.77] Li+
INFO:root:[ 1.93  0.01 13.4 ] Li+
INFO:root:[ 1.93  0.04 22.11] Li+
INFO:root:[ 1.93  0.01 30.75] Li+
INFO:root:[1.93 5.04 2.83] Li+
INFO:root:[ 1.93  5.24 11.67] Li+
INFO:root:[ 1.93  5.26 20.37] Li+
INFO:root:[ 1.93  5.26 29.08] Li+
INFO:root:[1.93 1.76 9.93] Li+
INFO:root:[ 1.93  1.77 18.63] Li+
INFO:root:[ 1.93  1.82 27.34] Li+
INFO:root:[1.93 6.95 8.18] Li+
INFO:root:[ 1.93  6.99 16.89] Li+
INFO:root:[ 1.93  7.01 25.58] Li+
INFO:root:[1.92 6.12 5.5 ] Li+
INFO:root:[ 1.93  6.11 14.28] Li+
INFO:root:[ 1.93  6.14 22.97] Li+
INFO:root:[ 1.93  6.14 31.72] Li+
INFO:root:[1.92 2.63 3.79] Li+
INFO:root:[ 1.93  2.62 12.54] Li+
INFO:root:[ 1.93  2.65 21.24] Li+
INFO:root:[ 1.93  2.64 30.01] Li+
INFO:root:[ 1.93  7.85 10.79] Li+
INFO:root:[ 1.93  7.87 19.5 ] L

Saving: events.json
Generating event kernal ...
Saving into: event_kernal.csv


'\ndef convert2(o):\n    if isinstance(o, np.int64): return int(o)\n    elif isinstance(o, np.int32): return int(o)  \n    try:\n        return o.as_dict()\n    except:\n        return o\n\n\nwith open("primitive_cell_neighbor.json","w") as f:\n    jsonstr=json.dumps(primitive_cell_cutoff_dict,indent=4,default=convert2)\n    f.write(jsonstr) \n'